In [6]:
%load_ext autoreload
%load_ext autotime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.42 ms (started: 2024-11-02 22:33:24 -07:00)


In [7]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 1.26 ms (started: 2024-11-02 22:33:24 -07:00)


In [8]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces/v1.0.5-develop/corona/cm1_1_48_20240926/cm1_1_48_20240926-COMPACT/*.pfw.gz"

time: 1.32 ms (started: 2024-11-02 22:33:24 -07:00)


In [9]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-deepspeed-dlio-step100-node-16"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
percentile = 0.5
run_dir = ".wisio/dftracer/deepspeed-dlio-step100-node-16"
# trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.6-develop/corona/megatron-deepspeed/dlio-scr-step100/node-16/v1/COMPACT/*.pfw.gz"
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.6-develop/corona/megatron-deepspeed/dlio-step100/node-16/v1/COMPACT/*.pfw.gz"


time: 1.28 ms (started: 2024-11-02 22:33:24 -07:00)


In [10]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.cluster import ExternalCluster
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"analyzer.time_granularity={52e6}",
            "cluster=external",
            "cluster.scheduler_address=tcp://127.0.0.1:41009",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    if isinstance(cluster, ExternalCluster):
        client = Client(cluster.scheduler_address)
    else:
        client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 2.58 s (started: 2024-11-02 22:33:25 -07:00)


In [11]:
client.dashboard_link

'http://127.0.0.1:8787/status'

time: 12.4 ms (started: 2024-11-02 22:33:27 -07:00)


In [12]:
client.restart()

<Client: 'tcp://127.0.0.1:41009' processes=32 threads=32>

time: 4.49 s (started: 2024-11-02 22:33:27 -07:00)


In [13]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    percentile=cfg.percentile,
    app_metrics=[
        'io_compute_ratio',
        'io_time',
        'ops',
        'unovlp_checkpoint_io_time',
        'unovlp_io_time',
        'unovlp_read_io_time',
    ],
    app_view_types=['proc_name', 'time_range', 'step'],
    posix_metrics=['time', 'iops'],
    posix_view_types=['proc_name', 'time_range'],
)

Additional high level metrics: ['app_io_time', 'checkpoint_io_time', 'compute_time', 'io_time', 'read_io_time']
Processing layer: APP
Categories: ['POSIX', 'STDIO', 'ai_framework', 'data_loader', 'reader', 'dlio_benchmark', 'checkpoint', 'storage']
Computing metric boundaries for APP layer: ['io_compute_ratio', 'io_time', 'ops', 'unovlp_checkpoint_io_time', 'unovlp_io_time', 'unovlp_read_io_time']
Processing layer: POSIX
Categories: ['POSIX', 'STDIO']
time: 4min 59s (started: 2024-11-02 22:33:34 -07:00)


In [16]:
output.handle_result([], result)

╭──────────────────────────────────────────── APP I/O Characteristics ────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime   8674.11 seconds                                                                                      │
│  App Time  App Time Breakdown                                                                                   │
│            ├── Compute Time: 7354.37 seconds (84.79%)                                                           │
│            ├── I/O Time: 944.20 seconds (10.89%)                                                                │
│            ├── Application I/O Time: 994.59 seconds (11.47%)                                                    │
│            ├── Read I/O Time: 3.41 seconds (0.04%)                                                              │
│            ├── Checkpoint I/O Time: 991.64 seconds (11.43%)                                                     │
│            ├── Unoverlapped Application Compute Time: 6579.34 seconds (75.85%)                                  │
│            ├── Unoverlapped Application I/O Time: 219.56 seconds (2.53%)                                        │
│            ├── Unoverlapped Compute Time: 6552.61 seconds (75.54%)                                              │
│            ├── Unoverlapped I/O Time: 142.44 seconds (1.64%)                                                    │
│            ├── Unoverlapped Checkpoint I/O Time: 219.56 seconds (2.53%)                                         │
│            └── Unoverlapped Read I/O Time: 0.00 seconds (0.00%)                                                 │
│                                                                                                                 │
╰─ R: Read - W: Write - M: Metadata  ─────────────────────────────────────────────────────────────────────────────╯
                                                                                                                   
╭─────────────────────────────────────────── POSIX I/O Characteristics ───────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          8674.11 seconds                                                                               │
│  I/O Time         784.83 seconds                                                                                │
│                   ├── Read - 3.24 seconds (0.41%)                                                               │
│                   ├── Write - 763.27 seconds (97.25%)                                                           │
│                   └── Metadata - 20.14 seconds (2.57%)                                                          │
│  I/O Operations   43,194,866 ops                                                                                │
│                   ├── Read - 1,970,432 ops (4.56%)                                                              │
│                   ├── Write - 551,136 ops (1.28%)                                                               │
│                   └── Metadata - 20,440,955 ops (47.32%)                                                        │
│  I/O Size         20.92 TiB                                                                                     │
│                   ├── Read - 13.67 GiB (0.06%)                                                                  │
│                   └── Write - 20.91 TiB (99.94%)                                                                │
│  Read Requests    4 kiB-1 MiB - 1,970,432 ops                                                                   │
│                   ├── <4 kiB - 6,904 ops (0.35%)                                                                │
│                   ├── 4-16 kiB - 172,434 ops (8.75%)  

time: 3.05 s (started: 2024-11-02 22:53:53 -07:00)


In [15]:
import dask
import dask.dataframe as dd

dask.compute(metric_boundaries['APP'])

NameError: name 'metric_boundaries' is not defined

time: 29.1 ms (started: 2024-11-02 22:53:41 -07:00)


In [ ]:
import dask

dask.compute((traces['te'].max() - traces['ts'].min()) / 1e6)

(8674.105622,)

time: 14.1 s (started: 2024-11-02 15:59:30 -07:00)


2024-11-02 16:50:27,669 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [13]:
(
    main_views['APP']
    .groupby(['proc_name', 'time_range'])
    .sum()
    .groupby(['time_range'])[
        [
            'app_io_time',
            'checkpoint_io_time',
            'compute_time',
            'io_time',
            'read_io_time',
            'time',
            'unovlp_app_compute_time',
            'unovlp_app_io_time',
            'unovlp_compute_time',
            'unovlp_io_time',
        ]
    ]
    .max()
    .sum()
    .compute()
)

app_io_time                  994.587904
checkpoint_io_time           991.639041
compute_time                7354.368991
io_time                      944.200664
read_io_time                   3.407638
time                       18980.771931
unovlp_app_compute_time     7354.368991
unovlp_app_io_time           994.587904
unovlp_compute_time         7354.368991
unovlp_io_time               944.200664
dtype: double[pyarrow]

time: 1.17 s (started: 2024-11-02 15:58:09 -07:00)


In [10]:
traces.head()

,func_id,cat,type,pid,tid,ts,te,time,tinterval,time_range,...,phase,size,hash,value,read_io_time,checkpoint_io_time,acc_pat,count,proc_name,io_cat
0,__lxstat64,POSIX,0,2837486,2837486,26180227,26180242,0.000015,<NA>,0,...,2,<NA>,<NA>,<NA>,0.0,0.0,0,1,app#corona202#2837486#2837486,3
1,__lxstat64,POSIX,0,2837486,2837486,26180252,26180269,0.000017,<NA>,0,...,2,<NA>,<NA>,<NA>,0.0,0.0,0,1,app#corona202#2837486#2837486,3
2,__xstat64,POSIX,0,2837486,2837486,26180292,26180654,0.000362,<NA>,0,...,2,<NA>,<NA>,<NA>,0.0,0.0,0,1,app#corona202#2837486#2837486,3
3,__lxstat64,POSIX,0,2837486,2837486,26180680,26180683,0.000003,<NA>,0,...,2,<NA>,<NA>,<NA>,0.0,0.0,0,1,app#corona202#2837486#2837486,3
4,__lxstat64,POSIX,0,2837486,2837486,26180694,26180697,0.000003,<NA>,0,...,2,<NA>,<NA>,<NA>,0.0,0.0,0,1,app#corona202#2837486#2837486,3


time: 3.03 s (started: 2024-11-02 14:09:16 -07:00)


In [11]:
len(traces[traces['checkpoint_io_time'].isna()])

0

time: 15.7 s (started: 2024-11-02 14:09:24 -07:00)


In [11]:
traces['checkpoint_io_time'].describe().compute()

count    1.346798e+08
mean     3.413595e-03
std      4.671081e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      8.897029e+01
Name: checkpoint_io_time, dtype: float64

time: 27.5 s (started: 2024-10-31 19:28:27 -07:00)


In [21]:
len(traces[traces['checkpoint_io_time'] > 0])

11520

time: 23.6 s (started: 2024-10-31 19:34:46 -07:00)


In [26]:
traces[traces['checkpoint_io_time'] > 0].compute()

,func_id,cat,type,pid,tid,ts,te,time,tinterval,time_range,...,phase,size,hash,value,read_io_time,checkpoint_io_time,proc_name,acc_pat,count,io_cat
13039,checkpoint_start_1_100,checkpoint,0,1466102,1466102,531105610,531120862,0.015252,<NA>,10,...,3,<NA>,<NA>,<NA>,0.0,0.015252,app#6875378042338841042#1466102#1466102,0,1,0
13047,checkpoint_end_1_100,checkpoint,0,1466102,1466102,545588363,596937429,51.349066,<NA>,10,...,3,<NA>,<NA>,<NA>,0.0,51.349066,app#6875378042338841042#1466102#1466102,0,1,0
13048,SCRPyTorchCheckpointing.checkpoint,checkpoint,0,1466102,1466102,531105578,596937521,65.831943,<NA>,10,...,3,<NA>,<NA>,<NA>,0.0,65.831943,app#6875378042338841042#1466102#1466102,0,1,0
1865,checkpoint_start_1_200,checkpoint,0,1466102,1466102,843882218,843885784,0.003566,<NA>,16,...,3,<NA>,<NA>,<NA>,0.0,0.003566,app#6875378042338841042#1466102#1466102,0,1,0
1873,checkpoint_end_1_200,checkpoint,0,1466102,1466102,856957353,911673721,54.716368,<NA>,16,...,3,<NA>,<NA>,<NA>,0.0,54.716368,app#6875378042338841042#1466102#1466102,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11537,checkpoint_end_1_2900,checkpoint,0,995356,995356,9370489589,9415520556,45.030967,<NA>,180,...,3,<NA>,<NA>,<NA>,0.0,45.030967,app#1847589366426109456#995356#995356,0,1,0
11538,SCRPyTorchCheckpointing.checkpoint,checkpoint,0,995356,995356,9352119461,9415520713,63.401252,<NA>,179,...,3,<NA>,<NA>,<NA>,0.0,63.401252,app#1847589366426109456#995356#995356,0,1,0
355,checkpoint_start_1_3000,checkpoint,0,995356,995356,9661980139,9662562299,0.58216,<NA>,185,...,3,<NA>,<NA>,<NA>,0.0,0.582160,app#1847589366426109456#995356#995356,0,1,0
364,checkpoint_end_1_3000,checkpoint,0,995356,995356,9679073131,9710024621,30.95149,<NA>,186,...,3,<NA>,<NA>,<NA>,0.0,30.951490,app#1847589366426109456#995356#995356,0,1,0


time: 24.4 s (started: 2024-10-31 19:39:03 -07:00)


In [24]:
traces['checkpoint_io_time'].nlargest(10).compute()

3492     88.970288
11939    88.969036
13847    88.955987
3760     88.954197
10629    88.950877
9057     88.950473
11933    88.950157
15040    88.946536
9838     88.944716
3610     88.940680
Name: checkpoint_io_time, dtype: float64

time: 38.8 s (started: 2024-10-31 19:36:46 -07:00)


In [22]:
traces.groupby(["time_range", "file_name"]).agg(
    {"compute_time": sum, "io_time": sum, "app_io_time": sum, "checkpoint_io_time": sum}
).groupby(["time_range"]).max().sum().compute() / 1e6

compute_time               0.0
io_time               0.001962
app_io_time                0.0
checkpoint_io_time         0.0
dtype: double[pyarrow]

time: 27.4 s (started: 2024-10-31 19:35:13 -07:00)


In [10]:
traces.groupby(["time_range", "host_name", "pid", "tid"]).agg(
    {"compute_time": sum, "io_time": sum, "app_io_time": sum}
).groupby(["time_range"]).max().sum().compute() / 1e6

compute_time    7342.517834
io_time         2338.373952
app_io_time     4514.236962
dtype: double[pyarrow]

time: 2.21 s (started: 2024-10-31 13:34:00 -07:00)


In [10]:
hlm.head()

,cat,io_cat,proc_name,step,acc_pat,time_range,func_id,app_io_time,checkpoint_io_time,compute_time,io_time,read_io_time,time,count,size_min,size_max,size
0,POSIX,3,app#16065659427268196451#2837486#2837486,0,0,3,close,0.0,0.0,0.0,0.002134,0.0,0.002134,1269,<NA>,<NA>,0
1,POSIX,3,app#16065659427268196451#2837486#2837486,0,0,3,open,0.0,0.0,0.0,0.016509,0.0,0.016509,1261,<NA>,<NA>,0
2,POSIX,0,app#16065659427268196451#2837486#2837486,0,0,3,readlink,0.0,0.0,0.0,0.000022,0.0,0.000022,1,<NA>,<NA>,0
3,POSIX,3,app#16065659427268196451#2837486#2839128,0,0,3,__xstat64,0.0,0.0,0.0,0.000346,0.0,0.000346,16,<NA>,<NA>,0
4,POSIX,3,app#16065659427268196451#2837486#2839590,0,0,3,__xstat64,0.0,0.0,0.0,0.000308,0.0,0.000308,16,<NA>,<NA>,0


time: 1.18 s (started: 2024-11-01 15:21:08 -07:00)


In [11]:
hlm['checkpoint_io_time'].describe().compute()

count    2.525459e+06
mean     3.482116e-02
std      9.124375e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.474955e+01
Name: checkpoint_io_time, dtype: float64

time: 685 ms (started: 2024-11-01 15:21:12 -07:00)


In [9]:
main_views['APP'].head()

app_io_time  \
time_range step proc_name                                               
5          2    app#10494875254371022816#1183618#1183618          0.0   
9          79   app#10494875254371022816#1183618#1183618          0.0   
10         98   app#10494875254371022816#1183618#1183618          0.0   
19         273  app#10494875254371022816#1183618#1183618          0.0   
           278  app#10494875254371022816#1183618#1183618          0.0   

                                                          checkpoint_io_time  \
time_range step proc_name                                                      
5          2    app#10494875254371022816#1183618#1183618                 0.0   
9          79   app#10494875254371022816#1183618#1183618                 0.0   
10         98   app#10494875254371022816#1183618#1183618                 0.0   
19         273  app#10494875254371022816#1183618#1183618                 0.0   
           278  app#10494875254371022816#1183618#1183618                 0.0   

                                                          compute_time  \
time_range step proc_name                                                
5          2    app#10494875254371022816#1183618#1183618           0.0   
9          79   app#10494875254371022816#1183618#1183618           0.0   
10         98   app#10494875254371022816#1183618#1183618           0.0   
19         273  app#10494875254371022816#1183618#1183618           0.0   
           278  app#10494875254371022816#1183618#1183618           0.0   

                                                          io_time  \
time_range step proc_name                                           
5          2    app#10494875254371022816#1183618#1183618      0.0   
9          79   app#10494875254371022816#1183618#1183618      0.0   
10         98   app#10494875254371022816#1183618#1183618      0.0   
19         273  app#10494875254371022816#1183618#1183618      0.0   
           278  app#10494875254371022816#1183618#1183618      0.0   

                                                          read_io_time  \
time_range step proc_name                                                
5          2    app#10494875254371022816#1183618#1183618           0.0   
9          79   app#10494875254371022816#1183618#1183618           0.0   
10         98   app#10494875254371022816#1183618#1183618           0.0   
19         273  app#10494875254371022816#1183618#1183618           0.0   
           278  app#10494875254371022816#1183618#1183618           0.0   

                                                              time  count  \
time_range step proc_name                                                   
5          2    app#10494875254371022816#1183618#1183618  2.465917      2   
9          79   app#10494875254371022816#1183618#1183618  2.452835      2   
10         98   app#10494875254371022816#1183618#1183618  2.451738      2   
19         273  app#10494875254371022816#1183618#1183618  2.452089      2   
           278  app#10494875254371022816#1183618#1183618  2.451779      2   

                                                          size_min  size_max  \
time_range step proc_name                                                      
5          2    app#10494875254371022816#1183618#1183618         0         0   
9          79   app#10494875254371022816#1183618#1183618         0         0   
10         98   app#10494875254371022816#1183618#1183618         0         0   
19         273  app#10494875254371022816#1183618#1183618         0         0   
           278  app#10494875254371022816#1183618#1183618         0         0   

                                                          size  \
time_range step proc_name                                        
5          2    app#10494875254371022816#1183618#1183618     0   
9          79   app#10494875254371022816#1183618#1183618     0   
10         98   app#10494875254371022816#1183618#1183618     0   
19         273  app#104948752543

time: 1.25 s (started: 2024-11-01 19:12:09 -07:00)


In [13]:
main_views['APP']['checkpoint_io_time'].describe().compute()

count    1.164220e+06
mean     7.553504e-02
std      1.342721e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.474955e+01
Name: checkpoint_io_time, dtype: float64

time: 788 ms (started: 2024-11-01 15:21:22 -07:00)


In [12]:
main_views['APP'].groupby(['time_range', 'proc_name']).agg(
    {"compute_time": sum, "io_time": sum}
).groupby(["time_range"]).max().sum().compute()

compute_time    7342.517834
io_time         2338.373952
dtype: float64

time: 571 ms (started: 2024-10-31 13:34:07 -07:00)


In [15]:
# traces.to_parquet("/usr/workspace/iopp/wisio_logs/_parquet/dftracer-deepspeed-dlio-scr-step100-node-16", write_metadata_file=True, compute=True)

time: 30.1 s (started: 2024-10-31 12:39:48 -07:00)


In [11]:
view_columns = views['APP'][('step',)].columns
view_time_columns = view_columns[['time' in col for col in view_columns]]
view_time_columns

Index(['app_io_time', 'checkpoint_io_time', 'compute_time', 'io_time',
       'read_io_time', 'time', 'unovlp_app_compute_time', 'unovlp_app_io_time',
       'unovlp_compute_time', 'unovlp_io_time', 'time_normalized',
       'unovlp_checkpoint_io_time', 'unovlp_checkpoint_io_time_per',
       'unovlp_io_time_per', 'unovlp_read_io_time', 'unovlp_read_io_time_per'],
      dtype='object')

time: 3.46 ms (started: 2024-11-01 19:12:31 -07:00)


In [12]:
app_time_range_view = views['APP'][('time_range',)].compute()
app_time_range_view[view_time_columns]

,app_io_time,checkpoint_io_time,compute_time,io_time,read_io_time,time,unovlp_app_compute_time,unovlp_app_io_time,unovlp_compute_time,unovlp_io_time,time_normalized,unovlp_checkpoint_io_time,unovlp_checkpoint_io_time_per,unovlp_io_time_per,unovlp_read_io_time,unovlp_read_io_time_per
time_range,,,,,,,,,,,,,,,,
5,0.064132,0.000000,24.424594,3.322201,0.064132,49.720059,24.424594,0.064132,24.424594,3.322201,0.052658,0.000000,0.000000,0.066818,0.0,0.0
9,0.020564,0.000000,51.291984,0.001214,0.020564,102.784045,51.291984,0.020564,51.291984,0.001214,0.108858,0.000000,0.000000,0.000012,0.0,0.0
10,33.179606,33.179606,19.539802,26.319338,0.010326,163.922179,19.539802,33.179606,19.539802,26.319338,0.173609,13.639804,0.083209,0.16056,0.0,0.0
19,0.020353,0.000000,51.291984,0.001217,0.020353,102.780838,51.291984,0.020353,51.291984,0.001217,0.108855,0.000000,0.000000,0.000012,0.0,0.0
23,0.020416,0.000000,51.291913,0.001205,0.020416,102.77842,51.291913,0.020416,51.291913,0.001205,0.108852,0.000000,0.000000,0.000012,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0.020572,0.000000,51.291973,0.001227,0.020572,102.779993,51.291973,0.020572,51.291973,0.001227,0.108854,0.000000,0.000000,0.000012,0.0,0.0
3,0.0,0.000000,0.0,6.037329,0.000000,121.081441,0.000000,0.000000,0.000000,6.037329,0.128237,0.000000,0.000000,0.049862,0.0,0.0
166,0.0,0.000000,0.0,0.867866,0.000000,59.994233,0.000000,0.000000,0.000000,0.867866,0.06354,0.000000,0.000000,0.014466,0.0,0.0


time: 1.62 s (started: 2024-11-01 19:12:35 -07:00)


In [13]:
app_step_view = views['APP'][('step',)].query('step > 0').compute()
app_step_view[view_time_columns].sort_values(['unovlp_app_io_time', 'unovlp_io_time'], ascending=False)

,app_io_time,checkpoint_io_time,compute_time,io_time,read_io_time,time,unovlp_app_compute_time,unovlp_app_io_time,unovlp_compute_time,unovlp_io_time,time_normalized,unovlp_checkpoint_io_time,unovlp_checkpoint_io_time_per,unovlp_io_time_per,unovlp_read_io_time,unovlp_read_io_time_per
step,,,,,,,,,,,,,,,,
1,0.057899,0.0,0.0,0.0,0.057899,89.478476,0.0,0.057899,0.0,0.0,0.094766,0.0,0.0,0.0,0.057899,0.000647
468,0.05747,0.0,0.0,0.0,0.057470,2.45185,0.0,0.057470,0.0,0.0,0.002597,0.0,0.0,0.0,0.057470,0.023439
421,0.022838,0.0,0.0,0.0,0.022838,2.452293,0.0,0.022838,0.0,0.0,0.002597,0.0,0.0,0.0,0.022838,0.009313
172,0.022363,0.0,0.0,0.0,0.022363,2.451611,0.0,0.022363,0.0,0.0,0.002596,0.0,0.0,0.0,0.022363,0.009122
103,0.021953,0.0,0.0,0.0,0.021953,2.452082,0.0,0.021953,0.0,0.0,0.002597,0.0,0.0,0.0,0.021953,0.008953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,0.0,0.0,0.0,0.0,0.000000,2.451768,0.0,0.000000,0.0,0.0,0.002597,0.0,0.0,0.0,0.000000,0.000000
2026,0.0,0.0,0.0,0.0,0.000000,2.451938,0.0,0.000000,0.0,0.0,0.002597,0.0,0.0,0.0,0.000000,0.000000
2590,0.0,0.0,0.0,0.0,0.000000,2.451767,0.0,0.000000,0.0,0.0,0.002597,0.0,0.0,0.0,0.000000,0.000000


time: 2.83 s (started: 2024-11-01 19:14:11 -07:00)


In [14]:
app_proc_name_view = views['APP'][('proc_name',)].compute()
app_proc_name_view[view_time_columns]

,app_io_time,checkpoint_io_time,compute_time,io_time,read_io_time,time,unovlp_app_compute_time,unovlp_app_io_time,unovlp_compute_time,unovlp_io_time,time_normalized,unovlp_checkpoint_io_time,unovlp_checkpoint_io_time_per,unovlp_io_time_per,unovlp_read_io_time,unovlp_read_io_time_per
proc_name,,,,,,,,,,,,,,,,
app#10494875254371022816#1183618#1183618,782.289733,782.289733,7327.13235,746.564448,0.000000,18264.53635,7327.13235,782.289733,7327.13235,746.564448,19.343914,0.0,0.0,0.040875,0.000000,0.000000
app#10494875254371022816#1183618#1184210,1.271435,0.000000,0.0,0.326765,1.271435,34.987654,0.00000,1.271435,0.00000,0.326765,0.037055,0.0,0.0,0.009339,1.271435,0.036340
app#10494875254371022816#1183618#1184777,1.300048,0.000000,0.0,1.241108,1.300048,35.550891,0.00000,1.300048,0.00000,1.241108,0.037652,0.0,0.0,0.034911,1.300048,0.036569
app#10494875254371022816#1183618#1185281,1.290237,0.000000,0.0,1.047079,1.290237,35.3699,0.00000,1.290237,0.00000,1.047079,0.03746,0.0,0.0,0.029604,1.290237,0.036478
app#10494875254371022816#1183618#1185630,1.299121,0.000000,0.0,1.176127,1.299121,35.295842,0.00000,1.299121,0.00000,1.176127,0.037382,0.0,0.0,0.033322,1.299121,0.036807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
app#16065659427268196451#2837484#2837709,0.0,0.000000,0.0,0.134704,0.000000,0.134704,0.00000,0.000000,0.00000,0.134704,0.000143,0.0,0.0,1.0,0.000000,0.000000
app#1995665909820648179#2211584#2211760,0.0,0.000000,0.0,0.15871,0.000000,0.15871,0.00000,0.000000,0.00000,0.158710,0.000168,0.0,0.0,1.0,0.000000,0.000000
app#12604783666688308289#2498468#2498648,0.0,0.000000,0.0,0.125706,0.000000,0.125706,0.00000,0.000000,0.00000,0.125706,0.000133,0.0,0.0,1.0,0.000000,0.000000


time: 1.33 s (started: 2024-11-01 19:14:16 -07:00)


- 1. mean(io_time) per time_range
- 2. max(io_time) per time_range
- 3. unoverlapped_checkpoint_io_time / io_time

In [15]:
bottlenecks['APP'][('proc_name',)]['io_time'].head()

,app_io_time,checkpoint_io_time,compute_time,io_time,read_io_time,time,count,size_min,size_max,size,...,unovlp_checkpoint_io_time_per,unovlp_io_time_per,unovlp_read_io_time,unovlp_read_io_time_per,io_compute_ratio_score,time_score,ops_score,unovlp_checkpoint_io_time_score,unovlp_io_time_score,unovlp_read_io_time_score
proc_name,,,,,,,,,,,,,,,,,,,,,
app#10494875254371022816#1183618#1183618,782.289733,782.289733,7327.13235,746.564448,0.0,18264.53635,144680,0,1875126300,180628053030,...,0.0,0.040875,0.0,0.0,low,critical,very high,none,very low,none
app#10494875254371022816#1183619#1183619,774.149271,774.149271,7326.663753,738.069855,0.0,18239.280398,144444,0,1875126300,180627993023,...,0.0,0.040466,0.0,0.0,low,critical,very high,none,very low,none
app#10494875254371022816#1183620#1183620,796.44335,796.443350,7326.655107,760.481635,0.0,18306.031846,144446,0,1875126300,180627993118,...,0.0,0.041543,0.0,0.0,low,critical,very high,none,very low,none
app#10494875254371022816#1183621#1183621,810.134904,810.134904,7326.70885,774.250137,0.0,18347.484308,144447,0,1875126300,180627993059,...,0.0,0.042199,0.0,0.0,low,critical,very high,none,very low,none
app#10494875254371022816#1183622#1183622,788.671181,788.671181,7327.176817,752.989259,0.0,18283.680892,144445,0,1875126300,180627993064,...,0.0,0.041184,0.0,0.0,low,critical,very high,none,very low,none


time: 1.28 s (started: 2024-11-01 19:14:21 -07:00)


In [16]:
bot_unovlp_checkpoint_io_time = bottlenecks['APP'][('step',)][
    'unovlp_checkpoint_io_time'
].compute()

time: 2.38 s (started: 2024-11-01 19:14:27 -07:00)


In [ ]:
bot_unovlp_checkpoint_io_time.sort_values('step')[
    list(
        sorted(
            set(view_time_columns).union(
                ['unovlp_checkpoint_io_time_per', 'unovlp_checkpoint_io_time_score']
            )
        )
    )
]

,app_io_time,checkpoint_io_time,compute_time,io_time,read_io_time,time,time_normalized,unovlp_app_compute_time,unovlp_app_io_time,unovlp_checkpoint_io_time,unovlp_checkpoint_io_time_per,unovlp_checkpoint_io_time_score,unovlp_compute_time,unovlp_io_time,unovlp_io_time_per,unovlp_read_io_time,unovlp_read_io_time_per
step,,,,,,,,,,,,,,,,,


time: 8.28 ms (started: 2024-11-01 19:14:34 -07:00)


2024-11-01 23:43:36,527 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [13]:
bot_unoverlapped_checkpoint_io_time[bot_unoverlapped_checkpoint_io_time['unoverlapped_checkpoint_io_time_score'] == "critical"].sort_values(by='unoverlapped_checkpoint_io_time_ratio', ascending=False).head(5)

,host_name,app_io_time,compute_time,io_time,time,count,size_min,size_max,size,read_io_time,...,time_normalized,unoverlapped_checkpoint_io_time,unoverlapped_checkpoint_io_time_ratio,unoverlapped_read_io_time,unoverlapped_read_io_time_ratio,io_compute_ratio_score,ops_score,time_score,unoverlapped_checkpoint_io_time_score,unoverlapped_read_io_time_score
time_range,,,,,,,,,,,,,,,,,,,,,
54,16161098942000804564,55.413276,0.0,47.153303,103.040947,2307424,110,53477376,877195818589,0.0,...,0.044065,55.960227,1.0,0.0,0.0,none,low,very low,critical,none
110,10466317107605025824,49.849793,0.0,38.464435,91.465113,2307837,107,53477376,878901953600,0.0,...,0.039115,53.511296,1.0,0.0,0.0,none,very low,very low,critical,none
0,17427840753905818416,0.000000,0.0,14.637338,159.656566,14559462,0,666660,7376069918,0.0,...,0.068277,144.782336,1.0,0.0,0.0,none,trivial,very low,critical,none
174,12428361473556904454,108.252772,0.0,48.631461,163.77136,2333672,64,1876171776,1642499402483,0.0,...,0.070036,115.191673,1.0,0.0,0.0,none,low,very low,critical,none
91,10466317107605025824,46.607407,0.0,31.396424,81.371762,2307885,107,53477376,878901941972,0.0,...,0.034798,50.466762,1.0,0.0,0.0,none,very low,very low,critical,none


time: 31.3 ms (started: 2024-10-31 14:45:04 -07:00)


In [18]:
len(traces[traces['file_name'].isna()])

20056190

time: 4.74 s (started: 2024-10-31 14:49:05 -07:00)


In [12]:
time_range_54 = traces.query('time_range == 54').compute()
time_range_54


,func_id,cat,type,pid,tid,ts,te,time,tinterval,time_range,...,file_name,host_name,phase,size,hash,value,proc_name,acc_pat,count,io_cat
16321,access,POSIX,0,995356,995356,2809796964,2809796988,0.000024,<NA>,54,...,6019635732081231872,1847589366426109456,2,<NA>,<NA>,<NA>,app#1847589366426109456#995356#995356,0,1,3
16322,__xstat,POSIX,0,995356,995356,2809797019,2809797029,0.00001,<NA>,54,...,6019635732081231872,1847589366426109456,2,<NA>,<NA>,<NA>,app#1847589366426109456#995356#995356,0,1,3
16323,access,POSIX,0,995356,995356,2809797062,2809797071,0.000009,<NA>,54,...,15488494264526061568,1847589366426109456,2,<NA>,<NA>,<NA>,app#1847589366426109456#995356#995356,0,1,3
16324,__xstat,POSIX,0,995356,995356,2809797078,2809797085,0.000007,<NA>,54,...,15488494264526061568,1847589366426109456,2,<NA>,<NA>,<NA>,app#1847589366426109456#995356#995356,0,1,3
16325,access,POSIX,0,995356,995356,2809797110,2809797118,0.000008,<NA>,54,...,4510474050500022272,1847589366426109456,2,<NA>,<NA>,<NA>,app#1847589366426109456#995356#995356,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15750,close,POSIX,0,995355,995355,2839201941,2839201943,0.000002,<NA>,54,...,8444630244888076288,1847589366426109456,2,<NA>,<NA>,<NA>,app#1847589366426109456#995355#995355,0,1,3
15751,fsync,POSIX,0,995355,995355,2839201948,2839201950,0.000002,<NA>,54,...,17078914073451841536,1847589366426109456,2,<NA>,<NA>,<NA>,app#1847589366426109456#995355#995355,0,1,3
15752,close,POSIX,0,995355,995355,2839201956,2839201957,0.000001,<NA>,54,...,17078914073451841536,1847589366426109456,2,<NA>,<NA>,<NA>,app#1847589366426109456#995355#995355,0,1,3
15753,fsync,POSIX,0,995355,995355,2839201963,2839201965,0.000002,<NA>,54,...,7783700778730591232,1847589366426109456,2,<NA>,<NA>,<NA>,app#1847589366426109456#995355#995355,0,1,3


time: 17.3 s (started: 2024-10-31 14:54:14 -07:00)


In [17]:
traces.query('time_range == 53 & func_id.str.contains("_1_800")').compute()

,func_id,cat,type,pid,tid,ts,te,time,tinterval,time_range,...,file_name,host_name,phase,size,hash,value,proc_name,acc_pat,count,io_cat
357,checkpoint_start_1_800,checkpoint,0,1466102,1466102,2776031328,2776045144,0.013816,<NA>,53,...,0,6875378042338841042,3,<NA>,<NA>,<NA>,app#6875378042338841042#1466102#1466102,0,1,0
365,checkpoint_end_1_800,checkpoint,0,1466102,1466102,2789873773,2864269207,74.395434,<NA>,53,...,0,6875378042338841042,3,<NA>,<NA>,<NA>,app#6875378042338841042#1466102#1466102,0,1,0
6929,checkpoint_start_1_800,checkpoint,0,1466103,1466103,2776031325,2776045144,0.013819,<NA>,53,...,0,6875378042338841042,3,<NA>,<NA>,<NA>,app#6875378042338841042#1466103#1466103,0,1,0
6937,checkpoint_end_1_800,checkpoint,0,1466103,1466103,2789853886,2864267397,74.413511,<NA>,53,...,0,6875378042338841042,3,<NA>,<NA>,<NA>,app#6875378042338841042#1466103#1466103,0,1,0
13504,checkpoint_start_1_800,checkpoint,0,1466104,1466104,2776031331,2776045146,0.013815,<NA>,53,...,0,6875378042338841042,3,<NA>,<NA>,<NA>,app#6875378042338841042#1466104#1466104,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5821,checkpoint_end_1_800,checkpoint,0,995354,995354,2793311652,2864333621,71.021969,<NA>,53,...,0,1847589366426109456,3,<NA>,<NA>,<NA>,app#1847589366426109456#995354#995354,0,1,0
3106,checkpoint_start_1_800,checkpoint,0,995355,995355,2776031288,2776045117,0.013829,<NA>,53,...,0,1847589366426109456,3,<NA>,<NA>,<NA>,app#1847589366426109456#995355#995355,0,1,0
3115,checkpoint_end_1_800,checkpoint,0,995355,995355,2793193577,2864272625,71.079048,<NA>,53,...,0,1847589366426109456,3,<NA>,<NA>,<NA>,app#1847589366426109456#995355#995355,0,1,0
401,checkpoint_start_1_800,checkpoint,0,995356,995356,2776031292,2776045118,0.013826,<NA>,53,...,0,1847589366426109456,3,<NA>,<NA>,<NA>,app#1847589366426109456#995356#995356,0,1,0


time: 12.6 s (started: 2024-10-31 14:56:44 -07:00)


In [16]:
time_range_54[time_range_54['func_id'].str.contains('checkpoint_end')]


,func_id,cat,type,pid,tid,ts,te,time,tinterval,time_range,...,file_name,host_name,phase,size,hash,value,proc_name,acc_pat,count,io_cat
14952,checkpoint_end_1_800,checkpoint,0,1733467,1733467,2809407311,2864258170,54.850859,<NA>,54,...,0,8274241731122146018,3,<NA>,<NA>,<NA>,app#8274241731122146018#1733467#1733467,0,1,0
5146,checkpoint_end_1_800,checkpoint,0,1733468,1733468,2809235843,2864289112,55.053269,<NA>,54,...,0,8274241731122146018,3,<NA>,<NA>,<NA>,app#8274241731122146018#1733468#1733468,0,1,0
11725,checkpoint_end_1_800,checkpoint,0,1733469,1733469,2809627815,2864312962,54.685147,<NA>,54,...,0,8274241731122146018,3,<NA>,<NA>,<NA>,app#8274241731122146018#1733469#1733469,0,1,0
1918,checkpoint_end_1_800,checkpoint,0,1733470,1733470,2809589301,2864298154,54.708853,<NA>,54,...,0,8274241731122146018,3,<NA>,<NA>,<NA>,app#8274241731122146018#1733470#1733470,0,1,0
8497,checkpoint_end_1_800,checkpoint,0,1733471,1733471,2809760901,2864255677,54.494776,<NA>,54,...,0,8274241731122146018,3,<NA>,<NA>,<NA>,app#8274241731122146018#1733471#1733471,0,1,0
1953,checkpoint_end_1_800,checkpoint,0,1733465,1733465,2809246345,2864297348,55.051003,<NA>,54,...,0,8274241731122146018,3,<NA>,<NA>,<NA>,app#8274241731122146018#1733465#1733465,0,1,0
8525,checkpoint_end_1_800,checkpoint,0,1733466,1733466,2808864180,2864277456,55.413276,<NA>,54,...,0,8274241731122146018,3,<NA>,<NA>,<NA>,app#8274241731122146018#1733466#1733466,0,1,0


time: 829 ms (started: 2024-10-31 14:56:26 -07:00)


In [17]:
hlm[(hlm['time_range'] == 54) & (hlm['func_id'].str.contains('checkpoint'))].compute()

,time_range,func_id,cat,io_cat,proc_name,host_name,acc_pat,app_io_time,compute_time,io_time,time,count,size_min,size_max,size,read_io_time,checkpoint_io_time
64034,54,checkpoint_end_1_800,checkpoint,0,app#8274241731122146018#1733467#1733467,8274241731122146018,0,54.850859,0.0,0.0,54.850859,1,<NA>,<NA>,0,0.0,54.850859
9707,54,checkpoint_end_1_800,checkpoint,0,app#8274241731122146018#1733471#1733471,8274241731122146018,0,54.494776,0.0,0.0,54.494776,1,<NA>,<NA>,0,0.0,54.494776
104182,54,checkpoint_end_1_800,checkpoint,0,app#8274241731122146018#1733466#1733466,8274241731122146018,0,55.413276,0.0,0.0,55.413276,1,<NA>,<NA>,0,0.0,55.413276
9273,54,checkpoint_end_1_800,checkpoint,0,app#8274241731122146018#1733470#1733470,8274241731122146018,0,54.708853,0.0,0.0,54.708853,1,<NA>,<NA>,0,0.0,54.708853
65010,54,checkpoint_end_1_800,checkpoint,0,app#8274241731122146018#1733469#1733469,8274241731122146018,0,54.685147,0.0,0.0,54.685147,1,<NA>,<NA>,0,0.0,54.685147
64446,54,checkpoint_end_1_800,checkpoint,0,app#8274241731122146018#1733468#1733468,8274241731122146018,0,55.053269,0.0,0.0,55.053269,1,<NA>,<NA>,0,0.0,55.053269
104489,54,checkpoint_end_1_800,checkpoint,0,app#8274241731122146018#1733465#1733465,8274241731122146018,0,55.051003,0.0,0.0,55.051003,1,<NA>,<NA>,0,0.0,55.051003


time: 522 ms (started: 2024-10-31 14:48:24 -07:00)


In [ ]:
bot_unoverlapped_checkpoint_io_time['io_bw'] = bot_unoverlapped_checkpoint_io_time['size'] / bot_unoverlapped_checkpoint_io_time['io_time']

In [16]:
bot_unoverlapped_read_io_time = bottlenecks['APP'][('time_range',)]['unoverlapped_read_io_time'].compute()
bot_unoverlapped_read_io_time

,host_name,app_io_time,compute_time,io_time,time,count,size_min,size_max,size,read_io_time,...,time_normalized,unoverlapped_checkpoint_io_time,unoverlapped_checkpoint_io_time_normalized,unoverlapped_read_io_time,unoverlapped_read_io_time_normalized,io_compute_ratio_score,ops_score,time_score,unoverlapped_checkpoint_io_time_score,unoverlapped_read_io_time_score
time_range,,,,,,,,,,,,,,,,,,,,,


time: 1.02 s (started: 2024-10-31 14:34:18 -07:00)


In [14]:
bottlenecks['APP'][('time_range',)]['ops'].head()

,host_name,app_io_time,compute_time,io_time,time,count,size_min,size_max,size,read_io_time,...,time_normalized,unoverlapped_checkpoint_io_time,unoverlapped_checkpoint_io_time_normalized,unoverlapped_read_io_time,unoverlapped_read_io_time_normalized,io_compute_ratio_score,ops_score,time_score,unoverlapped_checkpoint_io_time_score,unoverlapped_read_io_time_score
time_range,,,,,,,,,,,,,,,,,,,,,
53,6463326048822939753,169.367292,19.539788,30.552658,333.972451,130636,0,1875124100,765367867639,0.010219,...,0.142823,157.064223,0.067168,0.0,0.0,critical,critical,low,very low,none
109,14394617592516905742,162.011519,26.867272,30.965782,321.82492,183517,0,1875131607,763691207576,0.010396,...,0.137628,142.579055,0.060974,0.0,0.0,critical,critical,low,very low,none
113,8379356618791776904,0.025041,51.291824,0.017688,102.797803,301056,0,4051,179023488,0.025041,...,0.043961,0.000000,0.000000,0.0,0.0,none,very high,very low,none,none
19,8379356618791776904,0.020477,53.734354,0.001349,107.677211,315392,0,4051,183282176,0.020477,...,0.046048,0.000000,0.000000,0.0,0.0,none,high,very low,none,none
39,8379356618791776904,0.025003,51.292043,0.017688,102.798769,301056,0,4051,175867776,0.025003,...,0.043962,0.000000,0.000000,0.0,0.0,none,very high,very low,none,none


time: 1.1 s (started: 2024-10-31 13:34:16 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          338.02 seconds                                                                                │
│  I/O Time         35.27 seconds                                                                                 │
│                   ├── Read - 4.45 seconds (12.60%)                                                              │
│                   ├── Write - 0.04 seconds (0.10%)                                                              │
│                   └── Metadata - 30.96 seconds (87.77%)                                                         │
│  I/O Operations   25,820,495 ops                                                                                │
│                   ├── Read - 1,316,696 ops (5.10%)                                                              │
│                   ├── Write - 544,799 ops (2.11%)                                                               │
│                   └── Metadata - 23,462,488 ops (90.87%)                                                        │
│  I/O Size         8.15 GiB                                                                                      │
│                   ├── Read - 7.58 GiB (93.00%)                                                                  │
│                   └── Write - 584.08 MiB (7.00%)                                                                │
│  Read Requests    4 kiB-1 MiB - 1,316,170 ops                                                                   │
│                   ├── <4 kiB - 662,135 ops (50.31%)                                                             │
│                   ├── 4-16 kiB - 371,411 ops (28.22%)                                                           │
│                   ├── 16-64 kiB - 244,448 ops (18.57%)                                                          │
│                   ├── 64-256 kiB - 35,872 ops (2.73%)                                                           │
│                   └── 256 kiB-1 MiB - 2,304 ops (0.18%)                                                         │
│  Write Requests   4 kiB-64 kiB - 544,799 ops                                                                    │
│                   ├── <4 kiB - 535,197 ops (98.24%)                                                             │
│                   ├── 4-16 kiB - 130 ops (0.02%)                                                                │
│                   └── 16-64 kiB - 9,472 ops (1.74%)                                                             │
│  Nodes            16 nodes                                                                                      │
│                   ├── 39194 - 35.27 s (100.00%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,491 ops         │
│                   │   (6.25%)                                                                                   │
│                   ├── 12967 - 34.65 s (98.23%) - 485.23 MiB/37.31 MiB R/W (5.81/0.45%) - 1,617,918 ops (6.27%)  │
│                   ├── 35656 - 34.40 s (97.51%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,505 ops (6.25%)  │
│                   ├── 60580 - 33.30 s (94.39%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,514 ops (6.25%)  │
│                   ├── 51714 - 33.12 s (93.89%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,513 ops (6.25%)  │
│                   ├── 11383 - 32.50 s (92.13%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,503 ops (6.25%)  │
│                   ├── 23424 - 32.50 s (92.12%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,506 ops (6.25%)  │
│                   ├── 62039 - 32.47 s (92.05%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,514 ops (6.25%)  │
│                   ├── 33196 - 32.38 s (91.79%) - 485.2

time: 9min 56s (started: 2024-10-08 14:15:43 -07:00)
